<a href="https://colab.research.google.com/github/yinghuing/RAG_Agent_class/blob/master/chapter4/exercise2/exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習の準備
---

## 必要なライブラリのインストール

In [1]:
# !pip install -q langchain langchain-openai langchain-community langchain-core langchain-pinecone pypdf docx2txt wikipedia pinecone

In [2]:
!pip install -q langchain==0.3.7 langchain-openai==0.2.9 langchain-community==0.3.7 langchain-core==0.3.18 langchain-text-splitters==0.3.2 pypdf docx2txt wikipedia

In [3]:
!pip freeze | grep langchain

langchain==0.3.7
langchain-community==0.3.7
langchain-core==0.3.18
langchain-openai==0.2.9
langchain-pinecone==0.2.0
langchain-text-splitters==0.3.2


In [4]:
!pip install -q langchain-pinecone==0.2.0 pinecone==5.3.1

In [5]:
!pip freeze | grep pinecone

langchain-pinecone==0.2.0
pinecone==5.3.1
pinecone-client==5.0.1
pinecone-plugin-inference==1.1.0
pinecone-plugin-interface==0.0.7


## API キーの設定

*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、`LANGCHAIN_API_KEY`、`OPENAI_API_KEY`、`PINECONE_API_KEY` の 3 つを設定し、[**ノートブックからのアクセス**] を有効にします。
  *  `OPENAI_API_KEY` の [**値**] には指定されたキーを入力します。
  *  `LANGCHAIN_API_KEY` と `PINECONE_API_KEY` の [**値**] にはご自身で取得したキーを入力してください。
*  入力が完了したら、下のセルを実行します。

In [6]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

## サンプルファイルのアップロード

*  左ナビゲーションで [**ファイル**] アイコンをクリックします。
*  開いた [ファイル] 欄の何もない部分で右クリックし、[**新しいフォルダ**] をクリックします。(sample_data と同じ階層にフォルダが作られるようにします)
*  作成されたフォルダに **files** という名前を設定します。
*  files フォルダにカーソルを合わせ、3 点リーダアイコンをクリックして、[**アップロード**] をクリックします。
*  ローカルの files フォルダにあるすべてのファイルを選択してアップロードします。ご自身で用意したファイルをアップロードして使用しても構いません。
*  「警告」のポップアップが表示されますが問題ありません。[**OK**] をクリックしてポップアップを閉じます。

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Section 1: 関数の作成
---
RAG の各処理を実行する関数を作成します。  
タスク (Task) になっている各セルのコードの不足している部分を補完して処理を実装してください。

## Task: ドキュメントをロードする関数の作成
Python と LangChain フレームワークを使用して、PDF、Word (.docx)、およびテキスト (.txt) の各ドキュメントフォーマットに応じて動的にドキュメントをロードする関数を作成します。  
ファイル拡張子に基づいて適切な Document loader を選択し、対応するライブラリを使用してドキュメントをロードするコードを書いてください。  

* ファイル名から拡張子を抽出し、それに基づいて適切な Document loader を使用します
* 他の拡張子の場合は、対応していない旨のメッセージを表示して、None を返します
* 各ファイル形式に対応する Document loader を動的にインポートするコードを記述します。必要なインポートは、条件分岐の中で行います
* ロードプロセス中の状況を示すために、処理中のファイル名を `print` で表示します
* ドキュメントの内容をロードし、`data` 変数に格納して関数の出力として返します
  
参考：  
[langchain_community.document_loaders.text.TextLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.text.TextLoader.html)  
[langchain_community.document_loaders.pdf.PyPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)  
[langchain_community.document_loaders.word_document.Docx2txtLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.word_document.Docx2txtLoader.html)  
[langchain_community.document_loaders.wikipedia.WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html)  
https://python.langchain.com/docs/integrations/document_loaders/

In [8]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain_community.document_loaders.pdf import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain_community.document_loaders.word_document import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain_community.document_loaders.text import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

In [9]:
#wikipedia

def load_from_wikipedia(query, lang="en", load_max_docs=2):
  from langchain.document_loaders import WikipediaLoader
  loader = WikipediaLoader(query, lang, load_max_docs)
  data = loader.load()
  return data

## Task: ドキュメントをチャンクに分割する関数の作成
ロードしたドキュメント データを指定されたサイズにチャンク分割する関数を作成します。  

* Text splitter には `RecursiveCharacterTextSplitter` クラスを使用します
* `chunk_size` パラメータを使用して、各チャンクのサイズを指定します
* `chunk_overlap` パラメータを使用して、隣接するチャンク間の重複サイズを設定します
* これらのパラメータは関数の引数として指定でき、デフォルト値はそれぞれ `256` と `0` です
* インスタンス化した `RecursiveCharacterTextSplitter` を使用して、入力されたドキュメント データを分割します
* 関数は、分割されたチャンクを `chunks` というリストとして返します。
  
参考：  
[langchain_text_splitters.character.RecursiveCharacterTextSplitter](https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [10]:
def chunk_data(data, chunk_size=256, chunk_overlap=0):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    return chunks

## Task: Embedding を行い Vector store インスタンスを出力する関数の作成
ドキュメントのチャンクの embeddings を Pinecone の Index に挿入するか、既存の Index から embeddings を取得する関数を作成します。Index の存在チェック、インデックスの作成、embeddings の生成および挿入の処理を実装します。

* この関数は引数として、Index 名 (`index_name`) とチャンクのリスト (`chunks`) を受け取ります
* Vector store には Pinecone を使用します
* Embedding model には OpenAI の `text-embedding-3-small` を使用します。embedding のベクトル次元数は `1536` です
* Pinecone クライアントの `list_indexes()` メソッドを使用して、指定された `index_name` が既に存在するかどうかを確認します
* Index が存在する場合は、既存の Index から Pinecone の Vector store インスタンスを作成します
* Index が存在しない場合は、新しい Index を作成します。create_index メソッドを使用して、指定された次元数 (`1536`) とコサイン類似度 (`cosine`) を使用して Index を作成します
* チャンクの embeddings を生成し、それらを新しい Index に挿入し、Pinecone の Vector store インスタンスを作成します
* 最後に、Vector store インスタンス `vector_store` を関数の出力として返します

参考：  
[langchain_openai.embeddings.base.OpenAIEmbeddings](https://python.langchain.com/api_reference/openai/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)  
[langchain_pinecone.vectorstores.PineconeVectorStore](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html#pineconevectorstore)  
https://python.langchain.com/docs/integrations/vectorstores/pinecone/  


In [11]:
def insert_or_fetch_embeddings(index_name, chunks):
    # 必要なライブラリをインポート
    import pinecone
    from langchain_pinecone import PineconeVectorStore
    from langchain_openai import OpenAIEmbeddings
    from pinecone import PodSpec, ServerlessSpec

    # Pinecone クライアントを初期化
    pc = pinecone.Pinecone()

    # Embedding model のインスタンスを作成
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

    # embeddings の作成/ロード、Vector store の作成
    if index_name in pc.list_indexes().names():
        # Index がすでに存在する場合
        print(f'Index {index_name} already exists. Loading embeddings ... ')
        # Vector store インスタンスを作成
        vector_store = PineconeVectorStore.from_existing_index(index_name, embedding_model)
        print('Ok')
    else:
        # Index が存在しない場合
        print(f'Creating index {index_name} and embeddings ... ')

        # Index を作成
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

        # Vector store インスタンスを作成
        vector_store = PineconeVectorStore.from_documents(chunks, embedding_model, index_name=index_name)
        print('Ok')

    return vector_store

### Index を削除する関数

In [12]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()

    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

## Task: RAG Chain を実行する関数
---
会話履歴を考慮してユーザーの質問に回答する Chain を実行する関数を作成します。  
このタスクでは、Retriever の作成、会話履歴を考慮した質問の再構成、そして質問に対する適切な応答を生成する Chain を作成します。  

* この関数は引数として、`vector_store` 、ユーザーの質問 `q` 、Chat history のインスタンス `chat_history` 、セッション ID `session_id` (デフォルト値は `unused`) 、検索結果ドキュメントの取得数 `k` (デフォルト値は `20`) を受け取ります
* Chat model には OpenAI の `gpt-4o-mini` を使用します
* Retriever にも会話履歴を考慮させます
* LLM からの回答はテキストの形式に変換します
* `RunnableWithMessageHistory` を使用して Chain の処理に会話履歴を組み込みます
* Chain 処理の出力 `answer` を関数の出力として返します

参考：  
https://python.langchain.com/docs/concepts/#retrievers  


In [13]:
import langchain_openai

In [14]:
def get_answer_with_history(vector_store, q, chat_history, session_id='unused', k=20):
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain_openai import ChatOpenAI
    from operator import itemgetter
    from langchain_core.output_parsers import StrOutputParser
    from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
    from langchain_core.chat_history import BaseChatMessageHistory
    from langchain_core.runnables.history import RunnableWithMessageHistory
    from langchain.chains import create_history_aware_retriever

    # Chat model
    model = langchain_openai.ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # Retriever
    # パラメータは search_type='similarity', search_kwargs={'k': k} としてください
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":k})

    # 検索用にクエリを書き換えるためのプロンプト
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    # 会話履歴を考慮する Retriever を作成
    history_aware_retriever = create_history_aware_retriever(model, retriever, contextualize_q_prompt)


    # ユーザーのクエリに回答させるための Prompt template
    system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

    {context}"""
    human_message = "質問：{input}"

    chat_prompt = ChatPromptTemplate.from_messages([

        (
            "system", system_message
        ),
        MessagesPlaceholder("chat_history"),
        (
            "human", human_message
        )
    ])

    # ユーザーのクエリと history_aware_retriever が取得した Documents を出力する Runnable
    add_context = RunnablePassthrough.assign(context=history_aware_retriever)

    # Chain を定義
    rag_chain = add_context | chat_prompt | model | StrOutputParser()

    runnable_with_history = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: chat_history, # session_id を受け取って対応する chat message history インスタンス (BaseChatMessageHistory) を返す関数
        input_messages_key="input",
        history_messages_key="chat_history",


    )

    # Chain の実行
    answer = runnable_with_history.invoke({"input":q}, config={"configurable":{"session_id": session_id}})
    return answer

# Section 2: 関数を使用した処理の実行
---
Section 1 で作成した関数を使用して RAG の実行処理を実装します。
タスク (Task) になっている各セルのコードの不足している部分を補完して処理を実装してください。

## Task: ドキュメントのロードとチャンク化
* ファイルからドキュメントのデータをロードし、それをチャンクに分割します
* チャンクのサイズは `300` 、チャンク間の重複サイズは `0` とします

In [15]:
# ドキュメント データをロード
data = load_document("files/employment_regulations.pdf")


Loading files/employment_regulations.pdf


In [16]:
# チャンクに分割
chunks = chunk_data(data, chunk_size=300, chunk_overlap=0)

# 確認のため、チャンク数を表示する
print(len(chunks))

50


In [22]:
data = load_document("files/gingatetsudono_yoru.txt")
chunks += chunk_data(data, chunk_size=300, chunk_overlap=0)

In [23]:
print(len(chunks))

240


In [20]:
chunks

[Document(metadata={'source': 'files/employment_regulations.pdf', 'page': 0, 'text': 'LangChain Training 社 就業規定 \n第 1 章 総則 \n第 1 条 （目的） \n本規定は、LangChain Training 社（以下「当社」という）の従業員の就業に関する基\n本的な事項を定め、従業員の適正かつ円滑な業務運営を図ることを目的とします。 \n第 2 条 （適用範囲） \n本規定は、当社に勤務する全ての従業員に適用されます。 \n第 2 章 勤務時間・休憩時間 \n第 3 条 （勤務時間） \n1. 通常の勤務時間は、午前 9 時から午後 6 時までとします。 \n2. 所定の勤務時間は 1 日 8 時間、週 40 時間とします。 \n第 4 条 （休憩時間）'}, page_content='LangChain Training 社 就業規定 \n第 1 章 総則 \n第 1 条 （目的） \n本規定は、LangChain Training 社（以下「当社」という）の従業員の就業に関する基\n本的な事項を定め、従業員の適正かつ円滑な業務運営を図ることを目的とします。 \n第 2 条 （適用範囲） \n本規定は、当社に勤務する全ての従業員に適用されます。 \n第 2 章 勤務時間・休憩時間 \n第 3 条 （勤務時間） \n1. 通常の勤務時間は、午前 9 時から午後 6 時までとします。 \n2. 所定の勤務時間は 1 日 8 時間、週 40 時間とします。 \n第 4 条 （休憩時間）'),
 Document(metadata={'source': 'files/employment_regulations.pdf', 'page': 0, 'text': '1. 休憩時間は、勤務時間の途中に 1 時間とします。 \n2. 休憩時間は原則として、午後 12 時から午後 1 時までとします。 \n第 5 条 （残業） \n1. 業務上の必要により、勤務時間を超えて残業を命じる場合があります。 \n2. 残業を行う場合は、事前に上司の許可を得なければなりません。 \n第 3 章 休暇・休業 \n第 6 条 （年次有給休暇） \n1. 従業員は、入社後 6 

## 既存の Index を削除

In [17]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


## Task: Vector store インスタンスを取得
Index 名を指定して、その Vector store インスタンスを取得します

In [18]:
# Index 名を指定
index_name = 'askadocument'

# Vector store インスタンスを取得
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

Creating index askadocument and embeddings ... 
Ok


## Task: チャットボットを実行
* Chat history には `ChatMessageHistory` を使用します
* 必要な引数を `get_answer_with_history` 関数に渡して実行し、回答を表示します
* `get_answer_with_history` 関数に引数として渡す session_id は任意の英数字列で構いません

In [21]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

# Chat history
chat_history = ChatMessageHistory()

import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = get_answer_with_history(vector_store,q,chat_history,"b234")
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

Write Quit or Exit to quit.
Question #1: リモートワークの環境は？

Answer: リモートワーク中の勤務環境は、従業員が業務に適していることを確認し、安全かつ健康的な環境を維持する責任があります。また、会社は必要に応じてリモートワーク用のIT機器やソフトウェアを提供します。

 -------------------------------------------------- 

Question #2: 勤務時間はどうなりますか？

Answer: 通常の勤務時間は、午前9時から午後6時までと定められており、所定の勤務時間は1日8時間、週40時間です。リモートワーク中の勤務時間も通常の勤務時間と同様に扱われますが、柔軟な勤務時間を希望する場合は、上司の承認を得た上で調整が可能です。

 -------------------------------------------------- 

Question #3: その期間どんなサポートがありますか？

Answer: リモートワーク中の従業員に対しては、メンタルヘルスケアを重視し、定期的なフォローアップやカウンセリングの機会が提供されます。また、育児や介護など個別の事情に応じた柔軟な働き方を推奨し、必要に応じてリモートワーク用のIT機器やソフトウェアも提供されます。

 -------------------------------------------------- 

Question #4: quit
Quitting ... bye bye!
